In [ ]:
%%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

In [ ]:
!apt install unixodbc-dev
!pip install pyodbc

In [ ]:
import requests, six
import lxml.html as lh
from itertools import cycle, islice
from matplotlib import colors
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
%matplotlib inline

## Web Scraping

Extract the data from the website

In [ ]:
def web_scraping(key_words, years):

  final_df = []  
  
  for word in key_words:
    for year in years:
      a = word.split(" ")[0]
      b = word.split(" ")[1]

      url = "https://h1bdata.info/index.php?em=&job=" + a + "+" + b + "&city=&year=" + str(year) # URL to scrape data

      # Create a handle, page, to handle the contents of the website
      page = requests.get(url)

      # Store the contents of the website
      doc = lh.fromstring(page.content)

      # Parse data that are stored between <tr>.. </tr> tags
      tr_elements = doc.xpath('//tr')

      col = []
      i = 0

      for t in tr_elements[0]:
        i += 1
        name = t.text_content()
        # print(i, ".", name)
        col.append((name, []))  

      for j in range(1, len(tr_elements)):
        # T is our j'th row
        T = tr_elements[j]

        # Check if all the columns are present and add only that data
        if len(T) != 7:
          break
  
        # Ro
        i = 0

        for t in T.iterchildren():
          data = t.text_content()
          if i > 0:

            try:
              data = int(data)
            except:
              pass

          col[i][1].append(data)

          i += 1

      Dict = {title:column for (title, column) in col}
      df = pd.DataFrame(Dict)

      final_df.append(df)

  return pd.concat(final_df)

In [ ]:
key_words = ["Data Analyst", "Business Analyst", "Data Scientist", "Data Engineer"] # Job Roles to scrape
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020] # Years you want to scrape
data = web_scraping(key_words, years)

In [ ]:
data = data.reset_index().drop(columns = "index")

In [ ]:
data[['CITY', 'STATE', "r1"]] = data.LOCATION.str.split(", ", expand = True) # Expanding the location to state and city

In [ ]:
noise = {'NY': 'NY',
         'AR': 'AR',
         'MA': 'MA',
         'MD': 'MD',
         'CA': 'CA',
         'TX': 'TX',
         'VT': 'VT',
         'WA': 'WA',
         'WI': 'WI',
         'IL': 'IL',
         'GA': 'GA',
         'FL': 'FL',
         'MI': 'MI',
         'VA': 'VA',
         'PA': 'PA',
         'NE': 'NE',
         'NC': 'NC',
         'DC': 'DC',
         'OH': 'OH',
         'MN': 'MN',
         'IN': 'IN',
         'CO': 'CO',
         'CT': 'CT',
         'NJ': 'NJ',
         'TN': 'TN',
         'IA': 'IA',
         'MO': 'MO',
         'LA': 'LA',
         'SC': 'SC',
         'OK': 'OK',
         'PR': 'PR',
         'AZ': 'AZ',
         'AL': 'AL',
         'SAN RAMON,': 'CA',
         'DE': 'DE',
         'KY': 'KY',
         'NV': 'NV',
         'OR': 'OR',
         'KS': 'KS',
         'NH': 'NH',
         'MS': 'MS',
         'HI': 'HI',
         'CA 95134': 'CA',
         'UT': 'UT',
         'RI': 'RI',
         'NM': 'NM',
         'WY': 'WY',
         'ND': 'ND',
         'ME': 'ME',
         'ATLANTA': 'GA',
         'SD': 'SD',
         'MT': 'MT',
         'D.C.': 'DC',
         'WV': 'WV',
         'ID': 'ID',
         'MINNEAPOLIS': 'MN',
         'VI': 'VI',
         '': 'IN',
         'AK': 'AK',
         'ETS DRIVE': 'NJ',
         'MO 63105': 'MO',
         'WASHINGTON': 'WA',
         'DC 20006': 'DC',
         'IL 60654': 'IL'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cleaning the data
data["STATE"] = data["STATE"].map(noise)
data = data.drop(columns = 'r1')
data['CITY'] = data.CITY.str.replace(",", "")
data = data.drop(columns = 'LOCATION')
data['BASE SALARY'] = data['BASE SALARY'].str.replace(",", "")
data['BASE SALARY'] = data['BASE SALARY'].astype(int)
data.head()

,EMPLOYER,JOB TITLE,BASE SALARY,SUBMIT DATE,START DATE,CASE STATUS,CITY,STATE
0,VENIO LLC,DATA ANALYST,60000,02/15/2012,03/09/2012,WITHDRAWN,NEW YORK,NY
1,GENERAL DYNAMICS INFORMATION TECHNOLOGY INC,DATA ANALYST,62000,04/03/2012,09/15/2012,WITHDRAWN,LITTLE ROCK,AR
2,CATALYST ONLINE LLC,DATA ANALYST,77500,08/26/2013,09/09/2013,WITHDRAWN,NEWTON,MA
3,INOVALON INC,DATA ANALYST,80018,03/19/2013,09/18/2013,WITHDRAWN,BOWIE,MD
4,INTUIT INC,DATA ANALYST,90000,03/05/2013,08/26/2013,WITHDRAWN,MOUNTAIN VIEW,CA


In [ ]:
data.head()

,EMPLOYER,JOB TITLE,BASE SALARY,SUBMIT DATE,START DATE,CASE STATUS,CITY,STATE
0,VENIO LLC,DATA ANALYST,60000,02/15/2012,03/09/2012,WITHDRAWN,NEW YORK,NY
1,GENERAL DYNAMICS INFORMATION TECHNOLOGY INC,DATA ANALYST,62000,04/03/2012,09/15/2012,WITHDRAWN,LITTLE ROCK,AR
2,CATALYST ONLINE LLC,DATA ANALYST,77500,08/26/2013,09/09/2013,WITHDRAWN,NEWTON,MA
3,INOVALON INC,DATA ANALYST,80018,03/19/2013,09/18/2013,WITHDRAWN,BOWIE,MD
4,INTUIT INC,DATA ANALYST,90000,03/05/2013,08/26/2013,WITHDRAWN,MOUNTAIN VIEW,CA


In [ ]:
data['ID'] = data.index + 1 # Adding the ID to the data
data.columns = ['EMPLOYER', 'JOBTITLE', 'BASE_SALARY', 'SUBMIT_DATE', 'STARTING_DATE',
                'CASE_STATE', 'CITY', 'STATE', 'ID'] # Renaming the columns

In [ ]:
# Push the data into the server
def insert_to_db():
  server = 'SERVER_NAME'
  database = 'DATABASE_NAME'    
  username = 'USERNAME'    
  password = 'PASSWORD'

  conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)    
  cursor = conn.cursor()

  for index, row in data.iterrows():
    cursor.execute("INSERT INTO dbo.Ganapathy_Employment_data (ID, EMPLOYER, BASE_SALARY, SUBMIT_DATE, STARTING_DATE, CASE_STATE, CITY, STATE) VALUES(?,?,?,?,?,?,?,?)",
                   row.ID, row.EMPLOYER, row.BASE_SALARY, row.SUBMIT_DATE, row.STARTING_DATE, row.CASE_STATE, row.CITY, row.STATE)
  
  conn.commit()
  conn.close()

In [ ]:
insert_to_db()

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
!jupyter notebook --notebook-dir='/Content/drive/MyDrive/Colab Notebooks/'

The Jupyter HTML Notebook.

This launches a Tornado based HTML Notebook Server that serves up an
HTML5/Javascript Notebook client.

Subcommands
-----------

Subcommands are launched as `jupyter-notebook cmd [args]`. For information on
using subcommand 'cmd', do: `jupyter-notebook cmd -h`.

stop
    Stop currently running notebook server for a given port
password
    Set a password for the notebook server.
list
    List currently running notebook servers.

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--script
    DEPRECATED, IGNORED
--pylab
    DISABLED: use %pylab or %matplotlib in the notebook to enable matplotlib.
--debug
    set log level to logging.DEBUG (maximize logging output)
--no-browser
    Don't open the notebook in a browser after startup.
--allow-root
    Allow the notebook to be run from root user.
-y
    Answer y